<a href="https://colab.research.google.com/github/lavanyashukla/tick-tock/blob/main/LWM_1_2_Tick_Tock_%E2%80%93_Bug_or_not_%E2%80%93_Weave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Setup

In [5]:
!pip install openai -qq
!pip install weave -qq
from openai import OpenAI
from google.colab import userdata
from collections import Counter
import weave
from weave import weaveflow
import pandas as pd
import asyncio
import random
import csv
import json
import os
from weave.weaveflow import Model, Evaluation, Dataset

In [6]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
weave.init('tick-tock')

GraphClientTraceWithArtifactStorage()

In [8]:
!git clone https://github.com/lavanyashukla/tick-tock.git

Cloning into 'tick-tock'...
remote: Enumerating objects: 2555, done.
remote: Counting objects: 100% (2555/2555), done.
remote: Compressing objects: 100% (2539/2539), done.
remote: Total 2555 (delta 34), reused 2524 (delta 12), pack-reused 0
Receiving objects: 100% (2555/2555), 12.29 MiB | 7.10 MiB/s, done.
Resolving deltas: 100% (34/34), done.
Updating files: 100% (2493/2493), done.


In [9]:
# load the data
input_file = 'tick-tock/data/support_tickets_clean.csv'

# Open the CSV file for reading
with open(input_file, mode='r', encoding='utf-8') as csvfile:
    # Use csv.DictReader to read the CSV file into a dictionary
    reader = csv.DictReader(csvfile)

    # Initialize a list to store each row (as a dictionary)
    data = []

    # Iterate over the rows in the CSV file
    for row in reader:
        # Each row is a dictionary
        data.append(row)

# Now 'data' is a list of dictionaries, where each dictionary represents a row from the CSV
print(data[0])
print(len(data))

dataset = weaveflow.Dataset(data)
dataset_ref = weave.publish(dataset, 'tick-tock-dataset')

dataset = weaveflow.Dataset(data[:2])
dataset_ref = weave.publish(dataset, 'tick-tock-dataset-micro')

{'description': '10 out of 10\nComments: GrandMaster Training Data Platform\nChannel: In-app', 'raw_subject': 'NPS Response', 'subject': 'NPS Response', 'priority': 'low', 'problem_id': '', 'tags': "['nps_score', 'pendo_nps', 'personal', 'question']", 'id': '36652', 'question': 'question', 'customer_type': "['personal']", 'customer_type_2': '', 'type': 'nps_score'}
26590
Published Dataset to https://wandb.ai/llm-play/tick-tock/weaveflow/objects/tick-tock-dataset/versions/450adad76f7af89f0ac3
Published Dataset to https://wandb.ai/llm-play/tick-tock/weaveflow/objects/tick-tock-dataset-micro/versions/58d51bb88f89c2497563


In [10]:
!wandb login

wandb: Currently logged in as: lavanyashukla (llm-play). Use `wandb login --relogin` to force relogin


In [11]:
# Extracting the 'question' values where available
questions = [item.get('question', 'No question provided') for item in data]

# Counting occurrences of each unique question (or the placeholder 'No question provided')
question_counts = pd.Series(questions).value_counts().reset_index()
question_counts.columns = ['Question', 'Count']

# Display the counts in a table
print(question_counts.to_string(index=False))

            Question  Count
            question  21173
                none   2389
            type_bug   1536
type_feature_request   1492


In [12]:
# Initialize an empty set to store unique questions
unique_questions = set()

# Iterate over each dictionary in the data list
for item in data:
    # Check if 'question' key exists in the dictionary
    if 'question' in item:
        # Add the question to the set (sets automatically handle uniqueness)
        unique_questions.add(item['question'])

# Convert the set back to a list to get a list of unique questions
desired_tags = list(unique_questions)

print(desired_tags)

['type_bug', 'type_feature_request', 'none', 'question']


Steps:

- Navigate to the "Secrets" pane from the left navigation bar
- Add the OPENAI_API_KEY name and value
- Turn on the toggle of "Notebook access"

In [13]:
desired_tags = ["question", "none", "type_bug", "type_feature_request"]
filtered_data = [row for row in data if row.get('question') in desired_tags]
filtered_data = data

# Make a filtered dataset

In [14]:
# Let's look at the data
show_elem = 4
print(json.dumps(data[show_elem], indent=4))

{
    "description": "10 out of 10\nComments: No message provided.\nChannel: Email",
    "raw_subject": "NPS Response",
    "subject": "NPS Response",
    "priority": "low",
    "problem_id": "",
    "tags": "['nps_score', 'pendo_nps', 'question']",
    "id": "36643",
    "question": "question",
    "customer_type": "",
    "customer_type_2": "",
    "type": "nps_score"
}


In [15]:
count, score_priority, score_tags = 0, 0, 0
acc = {}

# Prompt Engineering

In [16]:
@weave.type()
class SupportTicketClassificationModel(Model):
    model_name: str = "gpt-3.5-turbo"
    system_prompt = " "

    @weave.op()
    async def predict(self, prompt, temperature=0) -> str:
        client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
        messages = [{"role": "user", "content": prompt}]
        response = client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            temperature=temperature,
        )
        extracted = response.choices[0].message.content
        return extracted


evalset_start_all, evalset_end_all = 500, 520

In [17]:
print(acc)

{}



# II. Basic Prompt + Evaluation

In [27]:
weave.init('tick-tock')
element = filtered_data[100]
ticket_text = element['description']

# Prompt – Classify Class
prompt = f"""
Classify the text delimited by triple backticks into one of the following classes.
Classes: {desired_tags}
Text: ```{ticket_text}```
Class: """

model = SupportTicketClassificationModel("gpt-3.5-turbo")
response = asyncio.run(model.predict(prompt))

# print("Prediction: "+response)

# if(response == element['question']):
#     score_tags += 1
#     print("Correct. Actual: "+element['question'])
# else:
#     print("Incorrect. Actual: "+element['question'])
# count += 1
# print()

evaluation = Evaluation(
    dataset, scores=[matchy_matchy, matchy_matchy2], example_to_model_input=example_to_model_input
)
# asyncio.run(evaluation.evaluate(model))
await evaluation.evaluate(model)

Exception in thread Thread-26 (_process_batches):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace_server/async_batch_processor.py", line 61, in _process_batches
    self.processor_fn(current_batch)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace_server/remote_http_trace_server.py", line 54, in _flush_calls
    r.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://trace.wandb.ai/call/upsert_batch


🍩 View call: https://wandb.ai/llm-play/tick-tock/weaveflow/calls/6075308e-221d-4ca8-85d6-5f4470e66481?convertToPeek=true


Output()

Output()

🍩 View call: https://wandb.ai/llm-play/tick-tock/weaveflow/calls/432534e4-42be-47f5-96de-1f0c616f8391?convertToPeek=true


{'matchy_matchy': {'correct': {'true_count': 0, 'true_fraction': 0.0}}}

In [18]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [19]:
@weave.op()
def matchy_matchy(example: dict, prediction: str) -> dict:
      return {'correct': example['question'] == prediction}

@weave.op()
def matchy_matchy2(example: dict, prediction: str) -> dict:
      return {'correct': example['question'] == prediction}

@weave.op()
def example_to_model_input(example: dict) -> str:
    return example["description"]

In [ ]:
def basic_prompt(evalset_start=400, evalset_end=500):
  count, score_tags = 0, 0
  for element in filtered_data[evalset_start:evalset_end]:
      # print(element)
      ticket_text = element['description']

      # Prompt – Classify Class
      prompt = f"""
      Classify the text delimited by triple backticks into one of the following classes.
      Classes: {desired_tags}
      Text: ```{ticket_text}```
      Class: """

      response = get_completion(prompt)
      model = SupportTicketClassificationModel()
      print(asyncio.run(model.predict(prompt)))

      print("Prediction: "+response)

      if(response == element['question']):
          score_tags += 1
          print("Correct. Actual: "+element['question'])
      else:
          print("Incorrect. Actual: "+element['question'])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count
acc['basic_prompt'] = basic_prompt(evalset_start_all, evalset_end_all)

# Add Examples

In [ ]:
# add examples to the prompt. pick examples randomly unless start is set.
# supports adding both random examples and a pre-defined set of examples
# evaluate on examples filtered_data[evalset_start:evalset_end]
def add_examples(num_examples=5, start=0, evalset_start=400, evalset_end=500, model="gpt-3.5-turbo", temperature=0
                 ):
  count, score_tags = 0, 0
  for element in filtered_data[evalset_start:evalset_end]:
      if start==0:
        # Ensure num_examples does not exceed the length of filtered_data
        num_samples = min(num_examples, len(filtered_data))

        # Randomly pick num_samples elements from filtered_data
        random_elements = random.sample(filtered_data, num_samples)
      else:
        random_elements = filtered_data[start:start+num_examples]

      examples = "".join([
          f"Text: ```{element['description']}```\nClass: {element.get('question', 'No question')}\n"
          for element in random_elements
      ])

      # print(element)
      ticket_text = element['description']

      # Prompt – Classify Class
      prompt = f"""
      Classify the text delimited by triple backticks into one of the following classes.
      Classes: {desired_tags}

      {examples}

      Text: ```{ticket_text}```
      Class: """
      response = get_completion(prompt, model=model, temperature=temperature)

      # print("Prompt: "+prompt)
      print("Prediction: "+response)

      if(response == element['question']):
          score_tags += 1
          print("Correct. Actual: "+element['question'])
      else:
          print("Incorrect. Actual: "+element['question'])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count
acc['add_5_examples'] = add_examples(5, 1000, evalset_start_all, evalset_end_all)

In [ ]:
acc['add_5_random_examples'] = add_examples(5, 0, evalset_start_all, evalset_end_all)

In [ ]:
acc['add_5_examples_gpt4'] = add_examples(5, 1000, evalset_start_all, evalset_end_all, model="gpt-4")

In [ ]:
acc['add_20_random_examples'] = add_examples(20, 0, evalset_start_all, evalset_end_all)

In [ ]:
acc['add_20_examples'] = add_examples(20,1000, evalset_start_all, evalset_end_all)

# Change Temperature

In [ ]:
acc['add_20_examples_change_temp'] = add_examples(20, 1000, evalset_start_all, evalset_end_all, temperature=0.7)

# Change model to GPT-4

In [ ]:
acc['add_20_examples_gpt4'] = add_examples(20, 1000, evalset_start_all, evalset_end_all, model="gpt-4")

# Describe Classes

In [ ]:
# add examples to the prompt. pick examples randomly unless start is set.
# supports adding both random examples and a pre-defined set of examples
# evaluate on examples filtered_data[evalset_start:evalset_end]
def describe_classes(num_examples=5, start=0, evalset_start=2500, evalset_end=2510, model="gpt-3.5-turbo", temperature=0):
  count, score_tags = 0, 0
  for element in filtered_data[evalset_start:evalset_end]:
      if start==0:
        # Ensure num_examples does not exceed the length of filtered_data
        num_samples = min(num_examples, len(filtered_data))

        # Randomly pick num_samples elements from filtered_data
        random_elements = random.sample(filtered_data, num_samples)
      else:
        random_elements = filtered_data[start:start+num_examples]

      examples = "".join([
          f"Text: ```{element['description']}```\nClass: {element.get('question', 'No question')}\n"
          for element in random_elements
      ])

      # print(element)
      ticket_text = element['description']

      # Prompt – Classify Class
      prompt = f"""
      Given the following description for each class:
      type_feature_request: A request for a feature by a user of Weights & Biases.
      type_bug: A bug report by a user of Weights & Biases.
      question: If the request is not related to Weights & Biases; or doesn't fit the above 2 categories.

      Classify the text delimited by triple backticks into one of the following classes.
      Classes: {desired_tags}

      {examples}

      Text: ```{ticket_text}```
      Class: """
      response = get_completion(prompt)

      # print("Prompt: "+prompt)
      print("Prediction: "+response)

      if(response == element['question']):
          score_tags += 1
          print("Correct. Actual: "+element['question'])
      else:
          print("Incorrect. Actual: "+element['question'])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  print()
  return score_tags/count
acc['add_5_examples_describe_classes'] = describe_classes(5,1000, evalset_start_all, evalset_end_all)
acc['add_20_examples_describe_classes'] = describe_classes(20,1000, evalset_start_all, evalset_end_all)

# Specify Steps

In [ ]:
# add examples to the prompt. pick examples randomly unless start is set.
# supports adding both random examples and a pre-defined set of examples
# evaluate on examples filtered_data[evalset_start:evalset_end]
def specify_steps(num_examples=5, start=0, evalset_start=2500, evalset_end=2510, examples=1, model="gpt-3.5-turbo", temperature=0):
  count, score_tags = 0, 0
  for element in filtered_data[evalset_start:evalset_end]:
      if start==0:
        # Ensure num_examples does not exceed the length of filtered_data
        num_samples = min(num_examples, len(filtered_data))

        # Randomly pick num_samples elements from filtered_data
        random_elements = random.sample(filtered_data, num_samples)
      else:
        random_elements = filtered_data[start:start+num_examples]

      examples = "".join([
          f"Text: ```{element['description']}```\nClass: {element.get('question', 'No question')}\n"
          for element in random_elements
      ])

      # print(element)
      ticket_text = element['description']

      # Prompt – Classify Class
      if examples:
        prompt = f"""
        Given the text delimited by triple backticks, perform the following actions:
        1 - Summarize what the user wants in 1-2 lines.
        2 - Recommend a next action based on the user' request.
        3 - Determine if the request is related to the product or company 'Weights & Biases'
        4 - Classify the into one of the following classes. Classes: {desired_tags}
        5 - Output a json object that contains the following keys: summary, recommended_action, is_wb, class

        Here are some examples help you with the classification step.
        {examples}

        And here's the text ```{ticket_text}```"""
      else:
        prompt = f"""
        Given the text delimited by triple backticks, perform the following actions:
        1 - Summarize what the user wants in 1-2 lines.
        2 - Recommend a next action based on the user' request.
        3 - Determine if the request is related to the product or company 'Weights & Biases'
        4 - Classify the into one of the following classes. Classes: {desired_tags}
        5 - Output a json object that contains the following keys: summary, recommended_action, is_wb, class

        Here's the text ```{ticket_text}```"""

      response_json = get_completion(prompt)
      # print("Prompt: "+prompt)
      print("Prediction: ")
      response = json.loads(response_json)
      print("Summary: "+response['summary'])
      print("Recommended Action: "+response['recommended_action'])
      print("Is W&B Related: "+str(response['is_wb']))
      print("Prediction: "+response['class'])

      if(response['class'] == element['question']):
          score_tags += 1
          print("Correct. Actual: "+element['question'])
      else:
          print("Incorrect. Actual: "+element['question'])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count

acc['specify_steps'] = specify_steps(5,1000, evalset_start_all, evalset_end_all)

In [ ]:
# acc['specify_steps_no_examples'] = specify_steps(5,1000, evalset_start_all, evalset_end_all, examples=0)

# Explain LLM Reasoning Behind Steps

In [ ]:
# add examples to the prompt. pick examples randomly unless start is set.
# supports adding both random examples and a pre-defined set of examples
# evaluate on examples filtered_data[evalset_start:evalset_end]
def specify_steps_explain_reasoning(num_examples=5, start=0, evalset_start=2500, evalset_end=2510, examples=1, model="gpt-3.5-turbo", temperature=0):
  count, score_tags = 0, 0
  for element in filtered_data[evalset_start:evalset_end]:
      if start==0:
        # Ensure num_examples does not exceed the length of filtered_data
        num_samples = min(num_examples, len(filtered_data))

        # Randomly pick num_samples elements from filtered_data
        random_elements = random.sample(filtered_data, num_samples)
      else:
        random_elements = filtered_data[start:start+num_examples]

      examples = "".join([
          f"Text: ```{element['description']}```\nClass: {element.get('question', 'No question')}\n"
          for element in random_elements
      ])

      # print(element)
      ticket_text = element['description']

      # Prompt – Classify Class
      if examples:
        prompt = f"""
        Given the text delimited by triple backticks, perform the following actions:
        1 - summary: Summarize what the user wants in 1-2 lines.
        2 - summary_reasoning: Explain your reasoning for the summary.
        3 - recommended_action: Recommend a next action based on the user' request.
        4 - recommended_action_reasoning: Explain your reasoning for the recommended next action.
        5 - is_wb: Determine if the request is related to the product or company 'Weights & Biases'.
        6 - is_wb_reasoning: Explain your reasoning for detemining if the request is W&B related.
        7 - class: Classify the into one of the following classes. Classes: {desired_tags}
        8 - Output a json object that contains the following keys: summary, summary_reasoning, recommended_action, recommended_action_reasoning, is_wb, is_wb_reasoning, class

        Here are some examples help you with the classification step.
        {examples}

        And here's the text ```{ticket_text}```.

        Make sure the output is only a json object.
        """
      else:
        prompt = f"""
        Given the text delimited by triple backticks, perform the following actions:
        1 - summary: Summarize what the user wants in 1-2 lines.
        2 - summary_reasoning: Explain your reasoning for the summary.
        3 - recommended_action: Recommend a next action based on the user' request.
        4 - recommended_action_reasoning: Explain your reasoning for the recommended next action.
        5 - is_wb: Determine if the request is related to the product or company 'Weights & Biases'.
        6 - is_wb_reasoning: Explain your reasoning for detemining if the request is W&B related.
        7 - class: Classify the into one of the following classes. Classes: {desired_tags}
        8 - Output a json object that contains the following keys: summary, summary_reasoning, recommended_action, recommended_action_reasoning, is_wb, is_wb_reasoning, class

        Here's the text ```{ticket_text}```"""

      response_json = get_completion(prompt)
      # print("Prompt: "+prompt)
      print("Prediction: ")
      response = json.loads(response_json)
      print("Summary: "+response['summary'])
      print("Summary Reasoning: "+response['summary_reasoning'])
      print("Recommended Action: "+response['recommended_action'])
      print("Recommended Reasoning: "+response['recommended_action_reasoning'])
      print("Is W&B Related: "+str(response['is_wb']))
      print("Is W&B Related Reasoning: "+response['is_wb_reasoning'])
      print("Prediction: "+response['class'])

      if(response['class'] == element['question']):
          score_tags += 1
          print("Correct. Actual: "+element['question'])
      else:
          print("Incorrect. Actual: "+element['question'])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count

acc['specify_steps_explain_reasoning'] = specify_steps_explain_reasoning(5,1000, evalset_start_all, evalset_end_all)

In [ ]:
acc['specify_steps_explain_reasoning_gpt4'] = specify_steps_explain_reasoning(5,1000, evalset_start_all, evalset_end_all, model="gpt-4")

# Prompt Chaining

In [ ]:
# add examples to the prompt. pick examples randomly unless start is set.
# supports adding both random examples and a pre-defined set of examples
# evaluate on examples filtered_data[evalset_start:evalset_end]
def prompt_chaining(num_examples=5, start=0, evalset_start=2500, evalset_end=2510, examples=1, model="gpt-3.5-turbo", temperature=0):
  count, score_tags = 0, 0
  for element in filtered_data[evalset_start:evalset_end]:
      if start==0:
        # Ensure num_examples does not exceed the length of filtered_data
        num_samples = min(num_examples, len(filtered_data))

        # Randomly pick num_samples elements from filtered_data
        random_elements = random.sample(filtered_data, num_samples)
      else:
        random_elements = filtered_data[start:start+num_examples]

      examples = "".join([
          f"Text: ```{element['description']}```\nClass: {element.get('question', 'No question')}\n"
          for element in random_elements
      ])

      # print(element)
      ticket_text = element['description']

      # Prompt – Classify Class
      prompt = f"""
      Given the text delimited by triple backticks, perform the following actions:
      1 - summary: Summarize what the user wants in 1-2 lines.
      2 - summary_reasoning: Explain your reasoning for the summary.
      3 - recommended_action: Recommend a next action based on the user' request.
      4 - recommended_action_reasoning: Explain your reasoning for the recommended next action.
      5 - is_wb: Determine if the request is related to the product or company 'Weights & Biases'.
      6 - is_wb_reasoning: Explain your reasoning for detemining if the request is W&B related.
      7 - Output a json object that contains the following keys: summary, summary_reasoning, recommended_action, recommended_action_reasoning, is_wb, is_wb_reasoning, class

      And here's the text ```{ticket_text}```.

      Make sure the output is only a json object.
      """

      response_json = get_completion(prompt)
      # print("Prompt: "+prompt)
      print("Prediction: ")
      response = json.loads(response_json)
      print("Summary: "+response['summary'])
      print("Summary Reasoning: "+response['summary_reasoning'])
      print("Recommended Action: "+response['recommended_action'])
      print("Recommended Reasoning: "+response['recommended_action_reasoning'])
      print("Is W&B Related: "+str(response['is_wb']))
      print("Is W&B Related Reasoning: "+response['is_wb_reasoning'])


      prompt_2 = f"""
        Given the following info about a user request:
        1 - text delimited by triple backticks: ```{ticket_text}```
        2 - summary of what the user wants: {response['summary']}
        3 - recommended next action based on the user' request: {response['recommended_action']}
        4 - whether the request is related to the product or company 'Weights & Biases': {response['is_wb']}

        Classify the text into one of the following classes. Classes: {desired_tags}

        Here are some examples help you with the classification step.
        {examples}

        Only print the name of the class"""
      response_class = get_completion(prompt_2)
      print("Prediction: "+response_class)

      if(response_class == element['question']):
          score_tags += 1
          print("Correct. Actual: "+element['question'])
      else:
          print("Incorrect. Actual: "+element['question'])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count

# acc['prompt_chaining'] = prompt_chaining(5,1000, evalset_start_all, evalset_end_all)

In [ ]:
df = pd.DataFrame(list(acc.items()), columns=['Prompt', 'Accuracy']).sort_values(by='Accuracy', ascending=True)
def highlight_max_accuracy(data, color='mediumpurple'):
    highlight = pd.DataFrame('', index=data.index, columns=data.columns)
    max_accuracy = data['Accuracy'] == data['Accuracy'].max()
    highlight[max_accuracy] = f'background-color: {color}'
    return highlight

# Applying the styling
df = df.style.apply(highlight_max_accuracy, axis=None)
df

# Next Steps
- [ ] Improve performance, make eval set more expansive
- [ ] Improve prompt chaining
- [ ] Improve summarization + suggest next steps using docs + support tickets (try RAG)
- [ ] How to evaluate the summary + next steps?
- [ ] RLHF for summary + next steps

Prompt  Accuracy
               basic_prompt      0.00
             add_5_examples      0.35
      add_5_random_examples      0.10
        add_5_examples_gpt4      0.30
     add_20_random_examples      0.60
            add_20_examples      0.90
add_20_examples_change_temp      0.80
       add_20_examples_gpt4      0.30
           describe_classes      0.40